In [3]:
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import itertools
#import enchant
from pylab import *
import re
import pickle
import sys
import csv

#csv.field_size_limit(sys.maxsize)
csv.field_size_limit(500 * 1024 * 1024)

from gensim import corpora, models, similarities


In [23]:
healthcare = pd.read_csv( "indeed_data.csv" )
jobs_titles=healthcare.iloc[:,1]
jobs_descriptions=healthcare.iloc[:,6]

In [45]:
allDat_ONET = pd.read_csv("result.csv")

In [26]:
from gensim import corpora, models, similarities, matutils
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import pickle
import re
import logging
from pandas import DataFrame, read_csv
from numpy import array, savetxt
import pylab as plt
import numpy as np
import matplotlib.pyplot
import math

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [66]:
input_file = 'indeed_data.csv'
file_save_weights_pickle = 'sigmoid_weights_'+input_file+'LDAmodel.pkl'
file_save_weights_excel = 'sigmoid_weights_'+input_file+'LDAmodel.xlsx'
from gensim import corpora, models, matutils


In [127]:
    from gensim import corpora, models, matutils

    # LOAD JOBS DATA
    jobsData = pd.read_csv(input_file)
    jobs_titles = jobsData['Role']
    jobs_descriptions = jobsData['Job_description']


    #We can create new tokenizer with python script CreatingDictionary.py
    tokenizer = pickle.load( open( "tokenizer.pkl", "rb" ) )

    # remove specific expressions
    documents=[]
    file = open('expressions_jobs.txt', 'r')
    expressions=[]
    lines=file.readlines()
    for line in lines:
        line=line.strip("\n").strip()
        line= line.split(",")
        expressions.append(line)
    expressions=list(itertools.chain(*expressions))

    for i in range(0, len(jobs_descriptions)):
        row=jobs_descriptions[i].lower()
        for exp in expressions:
                exp=str(exp).strip(" '' ")
                row = re.sub(exp, '', row).strip()
        row = re.sub(r'\w*\d\w*', '', row).strip()
        row=row.replace('\\t', ' ')
        documents.append(row.replace('\t', ' '))

    # remove words with specific prefixes
    texts=[]
    prefixes = ('www','detailswebsitehttp:', )
    for document in documents:
        d=document.split()
        for word in document.split():
            if word.startswith(prefixes):
             d.remove(word)
        texts.append(' '.join(d))

    # remove common words, stemming, punctuation  and tokenize
    #tokenize
    texts = [tokenizer.tokenize(text)
              for text in texts]

    #manual stoplist for all job datasets
    file = open('stoplist_manual.txt', 'r')
    stoplist_manual=[]
    lines=file.readlines()
    for line in lines:
        stoplist_manual.append(line.rsplit())
    stoplist_manual=list(itertools.chain(*stoplist_manual))
    texts = [[word for word in text if word not in stoplist_manual]
         for text in texts]

    # common english stoplist
    stoplist = stopwords.words('english')
    texts = [[word for word in text if word not in stoplist] for text in texts]  # remove words that appear only once
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
             frequency[token] += 1

    texts = [[token for token in text if frequency[token] > 1] #these two lines of code remove words that appear only once
              for text in texts]

    # Remove empty strings
    texts = [[word for word in text if word]
         for text in texts]

    stemmer = PorterStemmer()

    def stem_tokens(tokens, stemmer):
        stemmed_texts = []
        for text in texts:
         stemmed=[]
         for token in text:
            stemmed.append(stemmer.stem(token))
         stemmed_texts.append(stemmed)
        return stemmed_texts

    texts2 = stem_tokens(texts, stemmer)

In [119]:
documents=[]
expressions=['in order to', 'Check to ensure', 'All other']
for row in allDat_ONET['Description']:
    for exp in expressions:
        row=re.sub(exp,'', row)
    documents.append(row)

#tokenize
tokenizer = RegexpTokenizer(r'\w+')
texts = [tokenizer.tokenize(document.lower()) for document in documents]
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

# #Plot the frequency before
# df = DataFrame() #create empty dataframe
# for w in frequency:
#     #print w, frequency[w]
#     df = df.append({'Word': w, 'Frequency': frequency[w]}, ignore_index=True)
# df.to_excel('wordsOnet2018Before.xlsx')
# ax = df.plot.line()
# ax.axhline(y=df['Frequency'].mean(), xmin=-1, xmax=1, color='r', linestyle='--', lw=2, label='Mean')
# ax.axhline(y=math.sqrt(df['Frequency'].var()), xmin=-1, xmax=1, color='g', linestyle='--', lw=2, label='Standard Deviation')
# plt.rcParams.update({'font.size': 22})
# plt.legend(loc='upper left')
# plt.xlabel("Words")
# plt.ylabel("Frequency")
# plt.show()


# English stoplist
stoplist = stopwords.words('english')
texts = [[word for word in text if word not in stoplist] for text in texts]

# Manual stoplist
stoplist_manual = set('may usually separately often whose includes including primarily purposes necessary '
'using Using use Use ensure provide perform new according'.split())
texts = [[word for word in text if word not in stoplist_manual] for text in texts]
corpus = [dictionary.doc2bow(text) for text in texts]
print(len(corpus))


190


In [122]:

# Stemming
stemmer = PorterStemmer()
def stem_tokens(texts, stemmer):
    stemmed_texts = []
    for text in texts:
        stemmed=[]
        for token in text:
            stemmed.append(stemmer.stem(token))
        stemmed_texts.append(stemmed)
    return stemmed_texts


texts_f = stem_tokens(texts, stemmer)

# Create dictionary and Corpus and save them for later use
# mapping between the words from description and documents (jobs)
dictionary = corpora.Dictionary(texts_f)
# additional filtering removing words that appear in more than 90% of the occupations and less than 2 occupations
dictionary.filter_extremes(no_below=2, no_above=0.9)
dictionary.save('onet2018.dict') #1110 documents (jobs) x 4662 unique words

#print("Unique total words from 1110 job description: ", dictionary.items())
# #4662 integer id assigned to each unique word
#print dictionary.get(0) #first word in dictionary

#To actually convert tokenized documents to vectors:
corpus = [dictionary.doc2bow(text) for text in texts_f] #list of 1110 jobs x matching words with unique words and counting them
# The function doc2bow() simply counts the number of occurrences of each distinct word,
# converts the word to its integer word id (from the dictionary) and returns the result as a sparse vector.
# Save corpus - result is vector idword, number that word appears in particular job
corpora.MmCorpus.serialize('onet2018.mm', corpus)
#print (len(corpus[1]))



# load ONET dictionary
dictionary = corpora.Dictionary.load('onet2018.dict')
# load ONET corpus
corpus = corpora.MmCorpus('onet2018.mm')
print(len(corpus))

2020-08-18 03:14:34,034 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-08-18 03:14:34,118 : INFO : built Dictionary(1988 unique tokens: ['adjust', 'analysi', 'anesthesiologist', 'appli', 'applic']...) from 190 documents (total 33388 corpus positions)
2020-08-18 03:14:34,125 : INFO : discarding 151 tokens: [('counteract', 1), ('sent', 1), ('nutriti', 1), ('episod', 1), ('inhal', 1), ('amniot', 1), ('cytogenet', 1), ('marrow', 1), ('nomenclatur', 1), ('pair', 1)]...
2020-08-18 03:14:34,126 : INFO : keeping 1837 tokens which were in no less than 2 and no more than 171 (=90.0%) documents
2020-08-18 03:14:34,132 : INFO : resulting dictionary: Dictionary(1837 unique tokens: ['adjust', 'analysi', 'anesthesiologist', 'appli', 'applic']...)
2020-08-18 03:14:34,134 : INFO : saving Dictionary object under onet2018.dict, separately None
2020-08-18 03:14:34,138 : INFO : saved onet2018.dict
2020-08-18 03:14:34,191 : INFO : storing corpus in Matrix Market format to onet2018.mm
20

190


In [123]:
tfidf = models.TfidfModel(corpus, normalize=True) # step 1 -- initialize a model #normalize the resulting vectors to (Euclidean) unit length
corpus_tfidf = tfidf[corpus]
print(len(corpus_tfidf))

2020-08-18 03:14:45,113 : INFO : collecting document frequencies
2020-08-18 03:14:45,117 : INFO : PROGRESS: processing document #0
2020-08-18 03:14:45,150 : INFO : calculating IDF weights for 190 documents and 1837 features (19927 matrix non-zeros)


190


In [128]:
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=47, alpha='auto', eta='auto',  update_every=0, passes=5) # initialize an LSI transformation
lda.save('NewmodelLDA.lda')
vec_bow_jobs= [dictionary.doc2bow(text) for text in texts2] #bow_corpus #THIS IS JOBS CORPUS

2020-08-18 03:17:16,934 : INFO : using autotuned alpha, starting with [0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595, 0.021276595]
2020-08-18 03:17:16,938 : INFO : using serial LDA version on this node
2020-08-18 03:17:16,954 : INFO : running batch LDA training, 47 topics, 5 passes over the supplied corpus of 190 documents, updating model once every 190 documents, evaluating perplexity every 190 documents, iterating 50x with a convergence thresh

2020-08-18 03:17:20,972 : INFO : topic #19 (0.019): 0.001*"describ" + 0.001*"dna" + 0.001*"bone" + 0.001*"chorion" + 0.001*"chromosom" + 0.001*"congenit" + 0.001*"number" + 0.001*"label" + 0.001*"found" + 0.001*"microscop"
2020-08-18 03:17:20,975 : INFO : topic #29 (0.019): 0.014*"immunolog" + 0.007*"nasal" + 0.006*"allergi" + 0.004*"benefit" + 0.004*"nation" + 0.004*"intraderm" + 0.004*"peer" + 0.004*"differenti" + 0.003*"delay" + 0.003*"prognos"
2020-08-18 03:17:20,976 : INFO : topic #39 (0.032): 0.012*"skill" + 0.012*"materi" + 0.011*"client" + 0.010*"call" + 0.010*"home" + 0.009*"person" + 0.009*"meet" + 0.009*"imag" + 0.009*"train" + 0.009*"children"
2020-08-18 03:17:20,978 : INFO : topic #15 (0.037): 0.015*"social" + 0.013*"client" + 0.013*"custom" + 0.013*"healthcar" + 0.012*"servic" + 0.011*"inform" + 0.011*"worker" + 0.011*"data" + 0.010*"account" + 0.009*"comput"
2020-08-18 03:17:20,979 : INFO : topic #14 (0.045): 0.025*"patient" + 0.015*"treatment" + 0.013*"physic" + 0.011*"

In [129]:
lda.update(vec_bow_jobs)
vec_lda = lda[vec_bow_jobs]

2020-08-18 03:17:25,838 : INFO : running batch LDA training, 47 topics, 5 passes over the supplied corpus of 2642 documents, updating model once every 2642 documents, evaluating perplexity every 2642 documents, iterating 50x with a convergence threshold of 0.001000
2020-08-18 03:17:25,842 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2020-08-18 03:17:25,847 : INFO : PROGRESS: pass 0, at document #2000/2642
2020-08-18 03:17:27,398 : INFO : optimized alpha [0.037003726, 0.017681435, 0.02215004, 0.028718505, 0.059972618, 0.035691276, 0.017730428, 0.023238232, 0.020593824, 0.019789577, 0.0475398, 0.017196247, 0.019364882, 0.018204065, 0.116472155, 0.088828444, 0.067321114, 0.030786794, 0.021756597, 0.016867021, 0.023723178, 0.030198075, 0.02256005, 0.04208576, 0.022789642, 0.02674359, 0.017435536, 0.02453905, 0.020734953, 0.018510759, 0.029563151, 0.01834202, 0.018907145, 0.017108547, 0.027634092, 0.0173

2020-08-18 03:17:33,459 : INFO : topic #39 (0.565): 0.065*"time" + 0.043*"call" + 0.034*"busi" + 0.031*"assign" + 0.025*"role" + 0.025*"orient" + 0.025*"home" + 0.024*"help" + 0.023*"direct" + 0.023*"accur"
2020-08-18 03:17:33,460 : INFO : topic #15 (0.734): 0.050*"custom" + 0.048*"data" + 0.045*"servic" + 0.040*"provid" + 0.034*"healthcar" + 0.030*"inform" + 0.028*"insur" + 0.028*"use" + 0.026*"applic" + 0.026*"document"
2020-08-18 03:17:33,462 : INFO : topic #14 (1.067): 0.065*"patient" + 0.038*"medic" + 0.034*"posit" + 0.029*"perform" + 0.028*"clinic" + 0.026*"provid" + 0.025*"requir" + 0.021*"offic" + 0.021*"member" + 0.019*"need"
2020-08-18 03:17:33,463 : INFO : topic diff=0.214086, rho=0.475867
2020-08-18 03:17:33,474 : INFO : PROGRESS: pass 3, at document #2000/2642
2020-08-18 03:17:34,601 : INFO : optimized alpha [0.13797292, 0.015632225, 0.047997355, 0.09558952, 0.35398212, 0.1817941, 0.015528293, 0.05282778, 0.024304122, 0.026598785, 0.31892034, 0.016340282, 0.025625657, 0.02

In [130]:

    #LDA SIMILARITIES
    # ### Hellinger distance #lower means it is more similar
    from numpy import zeros
    _SQRT2 = np.sqrt(2)
    x = len(lda[vec_bow_jobs])  # number of data mined jobs
    y = len(lda[corpus])  # number of onet jobs
    simnew = zeros((x, y), dtype=float)
    print(x)
    print(y)
    
        
   

    

2642
190


In [132]:
for i in range(0, x):
        for j in range(0, y):
            dense1 = matutils.sparse2full(lda[vec_bow_jobs[i]], lda.num_topics)
            dense2 = matutils.sparse2full(lda[corpus[j]], lda.num_topics)
            sim = np.sqrt(np.sum((np.sqrt(dense1) - np.sqrt(dense2)) ** 2)) / _SQRT2
            simnew[i, j] = 1-sim
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [150]:
print(lda[vec_bow_jobs][2])

[(0, 0.018574793), (4, 0.02564312), (10, 0.06730525), (14, 0.18460014), (15, 0.22787157), (16, 0.08191257), (18, 0.0131946225), (22, 0.013844461), (23, 0.023720406), (24, 0.013168057), (25, 0.025508795), (30, 0.022732945), (34, 0.015127186), (39, 0.21613805)]


In [133]:

    sims = pd.DataFrame(simnew).transpose()

    #Standardization Armin applied both feature scaling and mean normalization
    sims = (sims - sims.mean()) / sims.std()

    #sum all weights to get one weight score for each job
    weights=sims.sum(axis=1)

    # Save occupations' weights
    weights=pd.DataFrame(np.hstack((pd.DataFrame(allDat_ONET['O*NET-SOC Code']), pd.DataFrame(weights))))
    print(weights)
    with open(file_save_weights_pickle, 'wb') as handle:
      pickle.dump(weights, handle)

              0        1
0    29-2055.00 -2166.44
1    29-2031.00   168.59
2    29-2032.00  1613.13
3    29-2035.00  869.676
4    29-2033.00    12.11
..          ...      ...
185  15-2041.02  280.928
186  53-2012.00  2275.37
187  11-3131.00  1180.04
188  43-4021.00  580.536
189  23-1022.00 -573.994

[190 rows x 2 columns]


In [136]:
import pyLDAvis.gensim

In [137]:
jd=pd.read_csv('Occupation Data.txt','\t')
desc_weights=pd.DataFrame(weights.sort_values(1, ascending=False)[:])
desc_weights=pd.DataFrame(pd.merge(desc_weights, jd, left_on=0, right_on='O*NET-SOC Code', how='inner'),columns=['O*NET-SOC Code', 'Title'])

writer = pd.ExcelWriter(file_save_weights_excel)
desc_weights.to_excel(writer,'Sheet1')
writer.save()
#LDA EVALUATION
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary, n_jobs=1)
pyLDAvis.save_html(vis, 'LDATopics43.html')
pyLDAvis.display(vis)

   



C:\Users\Admin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
 # split into train and test - random sample, but preserving order
train_size = int(round(len(corpus) * 0.8))
train_index = sorted(random.sample(xrange(len(corpus)), train_size))
test_index = sorted(set(xrange(len(corpus))) - set(train_index))
train_corpus = [corpus[i] for i in train_index]
test_corpus = [corpus[j] for j in test_index]
perplexity_value = []
logperp = []
coherence_values = []
x = []

number_of_words = sum(cnt for document in test_corpus for _, cnt in document)
parameter_list = range(20, 80, 3)
for parameter_value in parameter_list:
print "starting pass for parameter_value = %.3f" % parameter_value
model = models.LdaModel(corpus=train_corpus, id2word=dictionary, num_topics=parameter_value, alpha=0.01, eta=0.01, iterations=5)

log_perplex = model.log_perplexity(test_corpus)
perplex = np.exp2(-log_perplex)
print "Total Perplexity: %s" % perplex
x.append(parameter_value)
perplexity_value.append(perplex)
logperp.append(log_perplex)

coherencemodel = CoherenceModel(model=model, corpus=train_corpus, coherence='u_mass')
coherence_values.append(coherencemodel.get_coherence())


In [59]:
# Show graph
x = np.array (x)
y = np.array (perplexity_value)
plt.plot(x, y)
plt.xlabel("Num Topics")
plt.ylabel("Perplexity Value")
plt.show()

y = np.array (logperp)
plt.plot(x, y)
plt.xlabel("Num Topics")
plt.ylabel("Log Perplexity")
plt.show()

y = np.array(coherence_values)
plt.plot(x, y)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.show()



In [159]:
jd=pd.read_csv('Occupation Data.txt','\t')
desc_weights=pd.DataFrame(weights.sort_values(1, ascending=False)[:])
desc_weights=pd.DataFrame(pd.merge(desc_weights,jd,left_on=0, right_on='O*NET-SOC Code'))
writer = pd.ExcelWriter(file_save_weights_excel)
desc_weights.to_excel(writer,'Sheet2')
writer.save()

In [ ]:

df1=pd.read_csv("Health_Care_and_Social_Assistance.csv")
df2=pd.read_excel('WA.xlsx')
result=df1.merge(df2,on="O*NET-SOC Code")
result.to_csv("resultnext.csv")